---
jupyter: python3
toc: true
toc-depth: 3
number-sections: true
title: (05) 데이터 구조 변경
date: 2021-11-05
categories: pandas
author: limyj0708
comments:
  giscus:
    repo: quarto-dev/quarto-docs
format:
    html:
        page-layout: full
---

In [1]:
import pandas as pd
import numpy as np
import copy
from IPython.display import display_html, display

In [2]:
def display_multiple_dfs(dfs:list, styles, margin=10):
    display_target = ''
    for each_df in dfs:
        each_df_html = each_df[0].style.set_caption(f'<b>{each_df[1]}</b>').set_table_styles(styles).set_table_attributes(f"style='display:inline;margin:{margin}px'")._repr_html_()
        display_target += each_df_html
    display_html(display_target, raw = True)

In [3]:
styles = [
    {"selector" : "caption", "props" : "text-align:center; font-size:16px"}
]

# Pivot : 엑셀에서 보던 그것
`DataFrame.pivot(index=None, columns=None, values=None)`

- index : str or object or a list of str, optional
  - 새로운 프레임의 index로 사용할 컬럼
- columns : str of object or a list of str
  - 새로운 프레임의 컬럼으로 사용할 컬럼
- values : str, object or a list of the previous, optional
  - 새로운 프레임의 값을 계산하기 위해 사용하는 컬럼
  - 지정하지 않으면, 남아있는 모든 컬럼을 사용한다.

In [4]:
df = pd.DataFrame({'foo': ['one', 'one', 'one', 'two', 'two',
                           'two'],
                   'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'baz': [1, 2, 3, 4, 5, 6],
                   'zoo': ['x', 'y', 'z', 'q', 'w', 't']})
df

,foo,bar,baz,zoo
0,one,A,1,x
1,one,B,2,y
2,one,C,3,z
3,two,A,4,q
4,two,B,5,w
5,two,C,6,t


In [5]:
df.pivot(index='foo', columns='bar', values='baz')

bar,A,B,C
foo,,,
one,1,2,3
two,4,5,6


In [6]:
df.pivot(index='foo', columns='bar')

baz       zoo      
bar   A  B  C   A  B  C
foo                    
one   1  2  3   x  y  z
two   4  5  6   q  w  t

In [7]:
df.pivot(index='foo', columns='bar')['baz']

bar,A,B,C
foo,,,
one,1,2,3
two,4,5,6


In [8]:
df = pd.DataFrame({
       "lev1": [1, 1, 1, 2, 2, 2],
       "lev2": [1, 1, 2, 1, 1, 2],
       "lev3": [1, 2, 1, 2, 1, 2],
       "lev4": [1, 2, 3, 4, 5, 6],
       "values": [0, 1, 2, 3, 4, 5]})
df

,lev1,lev2,lev3,lev4,values
0,1,1,1,1,0
1,1,1,2,2,1
2,1,2,1,3,2
3,2,1,2,4,3
4,2,1,1,5,4
5,2,2,2,6,5


In [9]:
df.pivot(index="lev1", columns=["lev2", "lev3"] ,values="values")
# Multilevel Column
# 해당하는 조건에 맞는 값이 없으면 NaN이 들어가게 됨

lev2    1         2     
lev3    1    2    1    2
lev1                    
1     0.0  1.0  2.0  NaN
2     4.0  3.0  NaN  5.0

In [10]:
df.pivot(index=["lev1", "lev2"], columns=["lev3"],values="values")
# Multiindex

lev3         1    2
lev1 lev2          
1    1     0.0  1.0
     2     2.0  NaN
2    1     4.0  3.0
     2     NaN  5.0

In [11]:
#collapse-output
df.pivot(index=["lev1"], columns=["lev2"],values="values")
# 인덱스, 컬럼 쌍에 중복이 발생하면 에러가 출력됨
# ValueError: Index contains duplicate entries, cannot reshape

ValueError: Index contains duplicate entries, cannot reshape

# Pivot_table : Pivot의 확장 버전

`pandas.pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False, sort=True)`

In [28]:
#hide_input
df = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
                         "bar", "bar", "bar", "bar"],
                   "B": ["one", "one", "one", "two", "two",
                         "one", "one", "two", "two"],
                   "C": ["small", "large", "large", "small",
                         "small", "large", "small", "small",
                         "large"],
                   "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                   "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]})
df

,A,B,C,D,E
0,foo,one,small,1,2
1,foo,one,large,2,4
2,foo,one,large,2,5
3,foo,two,small,3,5
4,foo,two,small,3,6
5,bar,one,large,4,6
6,bar,one,small,5,8
7,bar,two,small,6,9
8,bar,two,large,7,9


In [29]:
table = pd.pivot_table(df, values='D', index=['A', 'B'], columns=['C'], aggfunc=np.sum)
table # aggfunc에 집계함수를 넣게 된다. 여기서는 총합

C        large  small
A   B                
bar one    4.0    5.0
    two    7.0    6.0
foo one    4.0    1.0
    two    NaN    6.0

In [30]:
table = pd.pivot_table(df, values='D', index=['A', 'B'],
                    columns=['C'], aggfunc=np.sum, fill_value=0)
table # fill_value에 할당된 값으로 NaN을 대체하게 됨

C        large  small
A   B                
bar one      4      5
    two      7      6
foo one      4      1
    two      0      6

In [31]:
table = pd.pivot_table(df, values=['D', 'E'], index=['A', 'C'],
                    aggfunc={'D': np.mean,
                             'E': np.sum})
table # aggfunc에 Dictionary를 할당하여 값마다 집계함수를 각각 다르게 설정할 수 있다.

D   E
A   C                  
bar large  5.500000  15
    small  5.500000  17
foo large  2.000000   9
    small  2.333333  13

In [32]:
table = pd.pivot_table(df, values=['D', 'E'], index=['A', 'C'],
                    aggfunc={'D': np.mean,
                             'E': [min, max, np.mean]})
table # 한 값에 여러 개의 집계함수 할당도 가능하다.

D   E              
               mean max      mean min
A   C                                
bar large  5.500000   9  7.500000   6
    small  5.500000   9  8.500000   8
foo large  2.000000   5  4.500000   4
    small  2.333333   6  4.333333   2

In [33]:
table = pd.pivot_table(df, values=['D', 'E'], index=['A', 'C'],
                    aggfunc={'D': np.mean,
                             'E': np.mean},
                    margins=True, margins_name="mean")
table # Values에 적용된 집계함수를 컬럼 전체에 적용한 행을 추가한다.
# 한 Value에 집계함수를 하나만 사용했을 때 적용 가능.
# margins_name을 지정하지 않으면 기본적으로 행 Index 이름은 All이 된다.

D         E
A    C                        
bar  large  5.500000  7.500000
     small  5.500000  8.500000
foo  large  2.000000  4.500000
     small  2.333333  4.333333
mean        3.666667  6.000000

# melt : Unpivot 하기

`pandas.melt(frame, id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None, ignore_index=True)`

- id_vars : tuple, list, or ndarray, optional
  - 식별자로 사용할 컬럼
- value_vars : tuple, list, or ndarray, optional
  - Unpivot 할 컬럼. 지정하지 않으면, id_vars에 할당되지 않은 모든 컬럼을 사용

In [34]:
df = pd.DataFrame({'A': {0: 'a', 1: 'b', 2: 'c'},
                   'B': {0: 1, 1: 3, 2: 5},
                   'C': {0: 2, 1: 4, 2: 6}})
df

,A,B,C
0,a,1,2
1,b,3,4
2,c,5,6


In [35]:
pd.melt(df, id_vars=['A'], value_vars=['B'])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5


In [36]:
pd.melt(df, id_vars=['A'], value_vars=['B', 'C'])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5
3,a,C,2
4,b,C,4
5,c,C,6


In [37]:
pd.melt(df, id_vars=['A'], value_vars=['B'],
        var_name='myVarname', value_name='myValname')
# 이름은 커스터마이징 가능

,A,myVarname,myValname
0,a,B,1
1,b,B,3
2,c,B,5


In [38]:
pd.melt(df, id_vars=['A'], value_vars=['B', 'C'], ignore_index=False)
# 원본 index 유지

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5
0,a,C,2
1,b,C,4
2,c,C,6


# sort_values : 정렬
`DataFrame.sort_values(by, *, axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last', ignore_index=False, key=None)`

- by : str, 혹은 str의 리스트
- axis : {0 or ‘index’, 1 or ‘columns’}, default 0
- ascending : bool, default True, True일 경우 오름차순
- inplace : bool, default False, 원본을 변경하는지
- kind : {‘quicksort’, ‘mergesort’, ‘heapsort’, ‘stable’}, default ‘quicksort’
  - 정렬 알고리즘 종류. 자세한 사항은 [numpy.sort](https://numpy.org/doc/stable/reference/generated/numpy.sort.html#numpy.sort) 참고
- na_position : {‘first’, ‘last’}, default ‘last’, NaN의 위치는 어디에 둘 지 선택
- ignore_index : bool, default False, If True, the resulting axis will be labeled 0, 1, …, n - 1.
- key : callable, optional
  - 정렬하기 전에 key에 할당된 함수를 값에 적용한다.
  - by에 할당된 컬럼들에 각각 적용된다.
  - input과 output의 크기가 같을 것이 요구된다. (정렬을 해야 하니, 당연한 이야기지만)
    - It should expect a Series and return a Series with the same shape as the input.

In [39]:
df_sample = pd.DataFrame({
    'col1': ['A', 'A', 'B', np.nan, 'D', 'C'],
    'col2': [2, 1, 9, 8, 7, 4],
    'col3': [0, 1, 9, 4, 2, 3],
    'col4': ['a', 'B', 'c', 'D', 'e', 'F']
})
df_sample

,col1,col2,col3,col4
0,A,2,0,a
1,A,1,1,B
2,B,9,9,c
3,NaN,8,4,D
4,D,7,2,e
5,C,4,3,F


In [40]:
df_sample.sort_values(by=['col1'])
# col1 기준 정렬

,col1,col2,col3,col4
0,A,2,0,a
1,A,1,1,B
2,B,9,9,c
5,C,4,3,F
4,D,7,2,e
3,NaN,8,4,D


In [41]:
df_sample.sort_values(by=['col1', 'col2'])
# col1, col2 기준 정렬

,col1,col2,col3,col4
1,A,1,1,B
0,A,2,0,a
2,B,9,9,c
5,C,4,3,F
4,D,7,2,e
3,NaN,8,4,D


In [42]:
df_sample.sort_values(by='col1', ascending=False, na_position='first')
# col1 기준, 내림차순, NaN을 상단에 두고 정렬

,col1,col2,col3,col4
3,NaN,8,4,D
4,D,7,2,e
5,C,4,3,F
2,B,9,9,c
0,A,2,0,a
1,A,1,1,B


In [43]:
col4 = df_sample.sort_values(by='col4')
#col4를 그냥 정렬하면, 대문자에서 소문자 순서로 정렬된다.
col4

,col1,col2,col3,col4
1,A,1,1,B
3,NaN,8,4,D
5,C,4,3,F
0,A,2,0,a
2,B,9,9,c
4,D,7,2,e


In [44]:
col4_key_lowercase = df_sample.sort_values(by='col4', key = lambda col: col.str.lower())
# col4의 각 알파벳들을 다 소문자로 치환 후 정렬하였다.
col4_key_lowercase

,col1,col2,col3,col4
0,A,2,0,a
1,A,1,1,B
2,B,9,9,c
3,NaN,8,4,D
4,D,7,2,e
5,C,4,3,F


# shift : Bigquery의 lead, lag에 대응되는 함수

In [45]:
df_sample = pd.DataFrame({
    'base_date': [  np.datetime64('2023-01-12')
                      , np.datetime64('2023-01-12')
                      , np.datetime64('2023-01-12')
                      , np.datetime64('2023-01-12')
                      , np.datetime64('2023-01-13')
                      , np.datetime64('2023-01-13')
                      
    ],
    'col2': [2, 1, 9, 8, 7, 4],
    'col3': [0, 1, 9, 4, 2, 3],
    'col4': ['a', 'B', 'c', 'D', 'e', 'F']
})
df_sample

,base_date,col2,col3,col4
0,2023-01-12,2,0,a
1,2023-01-12,1,1,B
2,2023-01-12,9,9,c
3,2023-01-12,8,4,D
4,2023-01-13,7,2,e
5,2023-01-13,4,3,F


In [46]:
df_sample['col2_shift_lag1'] = df_sample.groupby(['base_date'])['col2'].shift(1)
# base_date를 기준으로, col2에 대하여 shift 1 적용
# 즉, 이전 행의 col2 값을 출력

df_sample['col2_shift_lead1'] = df_sample.groupby(['base_date'])['col2'].shift(-1)
# base_date를 기준으로, col2에 대하여 shift -1 적용
# 즉, 다음 행의 col2 값을 출력

In [47]:
df_sample

,base_date,col2,col3,col4,col2_shift_lag1,col2_shift_lead1
0,2023-01-12,2,0,a,NaN,1.0
1,2023-01-12,1,1,B,2.0,9.0
2,2023-01-12,9,9,c,1.0,8.0
3,2023-01-12,8,4,D,9.0,NaN
4,2023-01-13,7,2,e,NaN,4.0
5,2023-01-13,4,3,F,7.0,NaN
